# EnergyNet Tutorial

Welcome to **EnergyNet**! This notebook will guide you through installing, setting up, and using this Python package for **smart grid simulation**.  

**Author:** Michael Wein  
**GitHub:** [energy-net](https://github.com/CLAIR-LAB-TECHNION/energy-net)  



#Setting up
Let's import the GitHub repository we’ll be working with: energy-net! The version we’re using here is publicly accessible and will only be updated with the changes from your branch once we’re certain everything is correct.

In [ ]:
%rm -rf energy-net # In case you have run this notebook in the past, and have an older version of energy-net
!git clone https://github.com/CLAIR-LAB-TECHNION/energy-net.git
%cd energy-net

In [ ]:
!pip install stable-baselines3[extra]


In [ ]:
import warnings
warnings.simplefilter("ignore") #To ignore warnings about your Python version or other silly things that clutter up the output.

# PCS Environment Guide


The `PCSEnv` class, located in the `pcs_env.py` file, is a **custom Gym environment** designed to simulate the behavior of a **PCS unit**. A PCS unit is composed of three types of components, which is what the name comes from:

- **Production units**: Components that generate energy (e.g., generators, solar panels, turbines).  
- **Consumption units**: Components that consume energy (e.g., electrical loads, machinery, appliances).  
- **Storage devices**: Components that store energy for later use (e.g., batteries, thermal storage tanks, capacitors).  

While the PCS unit can hold any number of units of each type, this environment is simplified to include **only a single consumption unit and a single storage unit** (with no production units).  

Now, you might be thinking: *“If there’s no production unit, where does the energy come from?”* Great question! In this environment, the PCS unit **buys energy from the grid (based on a set price curve)**, essentially an outside seller that can supply unlimited energy throughout the day at a variable price.  

This “price curve” will become more important later when we introduce an ISO (an entity that buys and sells energy). For now, it’s part of the environment dynamics and **cannot be controlled by the agent**.  

It’s also worth noting that the PCS unit **can sell energy back** to the grid. For simplicity, buying and selling prices are the same in this environment. The **only control the agent has** is deciding how much energy to buy or sell at each timestep.  

## Agent Goal / Reward

The agent’s performance is evaluated based on two main criteria:

- **Money**: The agent earns or loses money depending on when it buys and sells energy. Ideally, it should buy when prices are low and sell when prices are high to maximize profit over the day.  
- **Meeting energy demand**: The agent must provide enough energy for the consumption unit at every timestep. If it falls short, it receives a configurable negative reward.  

To make the task easier, the environment provides **predicted consumption data for the next timestep**, generated by an external model in `energy_net/consumption_prediction`.  

Let's start with the actual code!

##Setting up a basic environment

First, we create the environment. Here, we have the following parameters (we will cover more later):
- **render_mode**: for if you want the agent to (be able to) call env.render(), which provides information about what is happening at each time step. This is helpful for understanding what's happening in an episode, so we will use it for this simulation. In order to render, write "render_mode = 'human'" and write env.render() at each time step.
- **Data and prediction file path**: The prediction file predicts how much the consumption unit will consume at each step; the data file is how much is actually consumed.
- **Shortage Penalty**: this is how much the agent is punished for not having enough energy for a single time step. It should be chosen in accordance with the price for buying/selling energy.

In [ ]:
from energy_net.gym_envs.pcs_env import *
env = PCSEnv(render_mode='human', test_data_file='tests/gym/data_for_tests/synthetic_household_consumption_test.csv',
                 predictions_file='tests/gym/data_for_tests/consumption_predictions_without_features.csv',shortage_penalty = 1)

Here, and for the rest of this tutorial, we're going to treat one episode as one day where each interval or state is one half hour (in total, 48 states per day). It is possible to configure this fairly easily to be different (for example, for intervals to be much smaller or larger, and for episodes to be more than one day) using the environment parameters dt and episode_length_days, but for the purposes of this tutorial, we will use the default.

Every time we call env.reset(), it advances to the next day's data - if we hit the end of a data file, it will loop back to the beginning.

In [ ]:
# Reset at the start of each day.
# This clears financial metrics and ensures the strategy starts fresh for the date.
obs, info = env.reset()
env.render()

Here, we're going to take on a simple strategy - buy 10 units of energy for the first 24 time steps, and then sell 5 for the next 24 time steps.


*   Note - this will be restrained by how much it is actually possible to buy/sell. For example, if the PCS unit tries to buy 10 units of energy while energy is ≥ 90 (because 100 is the default max capacity here), then at the next timestep there will be 100 units in storage. Similarly, when it tries to sell more energy than it has, the environmnet will just sell the rest of the energy it does have and no more.


In [ ]:
print(f"Date: {env.current_datetime.strftime('%Y-%m-%d')}")

day_reward = 0.0
time_step = 0


#Intra-day Time Steps (48 steps per day, one for each half hour)
while True:
  # Generate a random battery action (Intent) between -10 and 10.
  # -10: Maximum Charge, 10: Maximum Discharge.
  action_value = 10 if time_step < 24 else -5
  action = np.array([action_value], dtype=np.float32)

  # Step the simulation.
  # The env will internally query the PriceCurveStrategy for the current price,
  # calculate the financial reward, and update the battery/consumption state.
  obs, reward, terminated, truncated, info = env.step(action)
  #Render all the details of the current time slot
  env.render()

  # Accumulate rewards (Net financial performance).
  day_reward += reward

  print(f"Step Reward: {reward:.2f}")

  # Check for End of Day
  # Terminated becomes True once the current_step reaches max_steps (48).
  if terminated or truncated:
      print(f"\n>>> Day Finished!")
      print(f">>> Total Day Reward: {day_reward:.2f}")
      print(f">>> Final Storage: {info['storage_after_units']:.2f} units")
      print(f">>> Total Shortages Today: {env.shortage_count}")
      break
  time_step+=1

## State Variables

- **Storage**  
  Energy stored in the battery at timestep t.

- **Consumption (Actual)**  
  Energy deducted during the transition from timestep t to t+1.

- **Current Price**  
  Cost of buying one unit of energy at timestep t.

- **Last Agent Action**  
  Energy bought or sold during timestep t-1.  
  **Note:** This only affects timestep t.


## How the Transition Works

Let’s look at a concrete example of an energy transition for better clarity.

### Example: Timestep 5

- Stored energy: 5 units  
- Energy price: 0.1 per unit  
- Consumption this timestep: 3 units  
- Agent buys: 1 unit (cost = 1 × 0.1 = 0.1)

**Step 1: Apply consumption**

$$
5 - 3 = 2 \quad \text{units remain in storage}
$$

**Step 2: Apply agent action (buy/sell)**

$$
2 + 1 = 3 \quad \text{units in storage at the start of timestep 6}
$$


## Important Insights

- **Consumption happens before buying energy**.  The agent must plan ahead:
  - To cover consumption at timestep t, ensure sufficient storage by timestep **t-1**.
  - Once t starts, it is **too late** to buy energy for that timestep.
  - Decisions are **anticipatory**, not reactive.


- Daily Battery Reset

  - In `PCSEnv`, the battery is reset to **0 units at the start of each day**. This encourages the agent to plan consumption and charging/discharging **within a single day**. No hoarding energy across days → focus on short-term optimization.

Now, let's train a real agent!

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
from stable_baselines3 import SAC
warnings.simplefilter("ignore") #To ignore warnings about your Python version or other silly things that clutter up the output.

In [ ]:
# ---------------- CONFIG ----------------
TRAIN_TIMESTEPS = 100000   # lower for quick tests; raise for serious training
EVAL_DAYS = 3              # how many days to evaluate after training
SEED = 42

# Create SAC model (device='auto' will use GPU in Colab if available)
model = SAC("MlpPolicy", env, verbose=0, seed=SEED, device="auto")

# Train
print("Starting training... (this may take a while depending on TRAIN_TIMESTEPS)")
model.learn(total_timesteps=TRAIN_TIMESTEPS)
print("Training finished.")


Next, let's evaluate how this function did! The code below will evaluate how the code did and produce various graphs. Feel free to test different algorithms against each other to see what works best here!

In [ ]:
# ---------------- EVALUATION ----------------
env_eval = PCSEnv(render_mode='human', test_data_file='tests/gym/data_for_tests/synthetic_household_consumption_test.csv',
                 predictions_file='tests/gym/data_for_tests/consumption_predictions_without_features.csv',shortage_penalty = 1)
# Prepare storage for metrics
timestamps = []
storage_before = []
storage_after = []
prices = []
actions = []
step_money = []
total_money = []
shortages = []
consumptions = []
avail_discharges = []

num_days = EVAL_DAYS
steps_per_day = env_eval.max_steps  # usually 48 for dt=0.5/24

for day in range(num_days):
    # reset returns (obs, info) in Gymnasium
    obs, info = env_eval.reset()
    done = False
    while not done:
        # predict action pass only the observation
        action, _state = model.predict(obs, deterministic=True)

        # Step env and unpack Gymnasium outputs
        next_obs, reward, terminated, truncated, info = env_eval.step(action)
        done = bool(terminated or truncated)

        # timestamp corresponding to the step (env.current_datetime has already advanced)
        ts = env_eval.current_datetime - timedelta(days=env_eval.dt)

        # Log metrics
        timestamps.append(ts)
        storage_before.append(info.get("storage_before_units"))
        storage_after.append(info.get("storage_after_units"))
        prices.append(info.get("current_price"))
        actions.append(float(info.get("battery_action", action[0] if hasattr(action, "__len__") else float(action))))
        step_money.append(info.get("step_money", 0.0))
        total_money.append(info.get("total_money_so_far", env_eval.get_money()))
        shortages.append(1 if info.get("shortage", False) else 0)
        consumptions.append(info.get("consumption_units", 0.0))
        avail_discharges.append(info.get("available_discharge_units", 0.0))

        obs = next_obs

print("Evaluation finished. Collected steps:", len(timestamps))

# Put into DataFrame
df = pd.DataFrame({
    "timestamp": timestamps,
    "storage_before": storage_before,
    "storage_after": storage_after,
    "price": prices,
    "action": actions,
    "step_money": step_money,
    "total_money": total_money,
    "shortage": shortages,
    "consumption": consumptions,
    "available_discharge": avail_discharges
})
df.head()


In [ ]:
# ---------- SIMPLE SINGLE PLOTS ----------
def line_plot(x, y, title, xlabel="timestep", ylabel=None, figsize=(10,3)):
    plt.figure(figsize=figsize)
    plt.plot(x, y)
    plt.title(title)
    plt.xlabel(xlabel)
    if ylabel:
        plt.ylabel(ylabel)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

# storage before/after
line_plot(df['timestamp'], df['storage_before'], "Storage (before step)", xlabel="time", ylabel="units")
line_plot(df['timestamp'], df['storage_after'],  "Storage (after step)",  xlabel="time", ylabel="units")

# price
line_plot(df['timestamp'], df['price'], "Price over time", xlabel="time", ylabel="price")

# actions
line_plot(df['timestamp'], df['action'], "Agent Actions over time", xlabel="time", ylabel="battery_intent")

# money (cumulative)
df['cumulative_money'] = df['step_money'].cumsum()
line_plot(df['timestamp'], df['cumulative_money'], "Cumulative Money over time", xlabel="time", ylabel="money")

# shortages as step plot
plt.figure(figsize=(12,2.5))
plt.plot(df['timestamp'], df['shortage'], drawstyle='steps-post')
plt.title("Shortages (0/1) over time")
plt.xlabel("time")
plt.ylabel("shortage")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# ---------- COMBINED MULTI-PANEL SUMMARY ----------
fig, axes = plt.subplots(4, 1, figsize=(14,12), sharex=True)
fig.suptitle("PCS Evaluation Summary", fontsize=16, fontweight='bold')

axes[0].plot(df['timestamp'], df['storage_after'])
axes[0].set_ylabel("Storage (units)")
axes[0].grid(alpha=0.3)

axes[1].plot(df['timestamp'], df['price'])
axes[1].set_ylabel("Price ($/unit)")
axes[1].grid(alpha=0.3)

axes[2].plot(df['timestamp'], df['action'])
axes[2].set_ylabel("Action (battery intent)")
axes[2].grid(alpha=0.3)

axes[3].plot(df['timestamp'], df['cumulative_money'])
axes[3].set_ylabel("Cumulative money ($)")
axes[3].set_xlabel("time")
axes[3].grid(alpha=0.3)

plt.tight_layout(rect=[0, 0.02, 1, 0.95])
plt.show()

# ISO Environment Guide

## What This Code Does

The `ISOEnv` is a **custom Gymnasium environment** designed for simulating electricity grid management scenarios. It represents a **day-ahead energy dispatch problem** where you need to make decisions about pricing and energy dispatch based on predicted consumption patterns.

### Core Concept

Imagine you're managing an electrical grid and need to:

1. **Predict** how much electricity households will consume tomorrow (you have forecasts)  
2. **Plan** how much power to dispatch and at what price  
3. **Evaluate** your decisions against what actually happened  

This environment simulates this scenario by:

- Loading **actual consumption data** (what really happened)  
- Loading **predicted consumption data** (your forecasts)  
- Allowing an agent to make decisions (prices and dispatch amounts)  
- Scoring performance based on how close the dispatch was to actual consumption  

### Key Features

**Automatic Feature Detection**: The environment automatically detects and includes any additional feature columns in your prediction CSV (like weather data, time features, etc.) beyond just the consumption forecast.

**Day-Based Episodes**: Each episode represents one full day, split into 48 half-hour time slots (30 minutes each).

**Observation Space**: For each day, you receive:

- 48 predicted consumption values (one per time slot)  
- Additional features for each time slot (if present in your data)  

**Action Space**: You must provide:

- 48 price values (what to charge per time slot)  
- 48 dispatch values (how much power to supply per time slot)  

**Performance Metric**: The environment uses **Mean Absolute Error (MAE)** between your dispatch and actual consumption as the cost function. Lower is better!

## How to Use This Environment

### Step 1: Prepare Your Data

You need two CSV files:

**Actual Consumption CSV** (`actual.csv`) and
**Predicted Consumption CSV** (`predictions.csv`)
Let's initialize the environment!



In [ ]:
from energy_net.gym_envs.iso_env import *
warnings.simplefilter("ignore")


env = ISOEnv(
    actual_csv='tests/gym/data_for_tests/synthetic_household_consumption_test.csv',
    predicted_csv='tests/gym/data_for_tests/consumption_predictions_without_features.csv',
    steps_per_day=48  # 48 half-hour slots = 24 hours
)


Next, let's run a basic simulation.

In [ ]:
import numpy as np

# Reset to get the first day's observation
obs, info = env.reset()

# The observation contains predictions + features
# Let's extract just the predictions (first 48 values)
predictions = obs[:48]

# Create a simple action: prices and dispatch amounts
prices = np.linspace(0.1, 0.5, 48)  # Prices from $0.10 to $0.50
dispatch = predictions  # Dispatch exactly what we predicted

# Combine into action: [48 prices, 48 dispatch values]
action = np.concatenate([prices, dispatch])

# Take a step (simulate one day)
next_obs, reward, terminated, truncated, step_info = env.step(action)

# Visualize the results
env.render()


### Understanding the Output

When you call `env.render()`, you'll see:

- **Time slots**: Each row shows a 30-minute period
- **CSV Row**: Which row in your original data this corresponds to
- **Pred vs Actual**: Forecasted vs actual consumption
- **Dispatch**: How much power you decided to supply
- **Price**: The price you set
- **Features**: Any additional columns (weather, time features, etc.)
- **MAE (Mean Absolute Error)**: How accurate your dispatch was


Now, let's try training an agent!

In [ ]:
from stable_baselines3 import SAC

# Train an agent
model = SAC(
    "MlpPolicy",
    env,
    gamma=0.0,       # <-- critical
    learning_starts=0,
    train_freq=1,
    gradient_steps=1000,
    verbose=0,
)
model.learn(total_timesteps=3)



Now, let's graph the output of this learning process! Don't expect such great results yet - since this ISO is acting without the input of a PCS unit consuming energy, we had to give it a fairly bogus reward. In the alternating environment, it will have a better reward function.  

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get the last day's data from the environment's render record
# Run this cell after your training/testing loop

# First, let's capture data during the test loop
obs, info = env.reset()
day_records = []

for day in range(10):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    record = env.render()

    # Store the record for plotting
    if record is not None:
        day_records.append({
            'actual': info['realized'],
            'dispatch': info['dispatch'],
            'prices': info['prices'],
            'timestamp': info['timestamp']
        })

    if terminated:
        obs, info = env.reset()

# Now plot the last day (or any specific day)
day_to_plot = -1  # -1 for last day, 0 for first day, etc.

if day_records:
    data = day_records[day_to_plot]

    # Create time steps (48 half-hour intervals)
    timesteps = np.arange(len(data['actual']))

    # Create figure with all three metrics
    fig, ax1 = plt.subplots(figsize=(14, 7))

    # Plot actual consumption and dispatch on left y-axis
    ax1.set_xlabel('Time Step (30-min intervals)', fontsize=12)
    ax1.set_ylabel('Consumption / Dispatch (kWh)', fontsize=12, color='black')

    line1 = ax1.plot(timesteps, data['actual'],
                     color='#FF1744', linewidth=2.5,
                     label='Actual Consumption', marker='o', markersize=4)
    line2 = ax1.plot(timesteps, data['dispatch'],
                     color='#00E676', linewidth=2.5,
                     label='Model Dispatch', marker='s', markersize=4, linestyle='--')

    ax1.tick_params(axis='y', labelcolor='black')
    ax1.grid(True, alpha=0.3)

    # Create second y-axis for prices
    ax2 = ax1.twinx()
    ax2.set_ylabel('Price ($/kWh)', fontsize=12, color='#2979FF')

    line3 = ax2.plot(timesteps, data['prices'],
                     color='#2979FF', linewidth=2.5,
                     label='ISO Price', marker='^', markersize=4, linestyle=':')

    ax2.tick_params(axis='y', labelcolor='#2979FF')

    # Combine legends
    lines = line1 + line2 + line3
    labels = [l.get_label() for l in lines]
    ax1.legend(lines, labels, loc='upper left', fontsize=11, framealpha=0.9)

    # Title with date
    plt.title(f'ISO Environment - Day Analysis\n{data["timestamp"].strftime("%Y-%m-%d")}',
              fontsize=14, fontweight='bold', pad=20)

    plt.tight_layout()
    plt.show()

    # Print summary statistics
    mae = np.mean(np.abs(data['dispatch'] - data['actual']))
    avg_price = np.mean(data['prices'])

    print(f"\n{'='*50}")
    print(f"Day Summary for {data['timestamp'].strftime('%Y-%m-%d')}")
    print(f"{'='*50}")
    print(f"Mean Absolute Error (MAE): {mae:.4f} kWh")
    print(f"Average Price Set:         ${avg_price:.4f}/kWh")
    print(f"Total Actual Consumption:  {np.sum(data['actual']):.2f} kWh")
    print(f"Total Dispatch:            {np.sum(data['dispatch']):.2f} kWh")
    print(f"{'='*50}\n")
else:
    print("No day records available. Make sure to run the test loop first.")

## Important Notes on ISO Environment

- **One step = One day**: Each call to `step()` processes an entire day (48 time slots)  
- **Automatic loop**: When the environment reaches the end of your data, it automatically loops back to the beginning  
- **Reward**: Negative MAE (so maximizing reward = minimizing error)  
- **Sequential days**: The environment automatically advances through your dataset day by day


# Alternating ISO-PCS

## What This Code Does

Now we get to the **crux of EnergyNet - a multi agent environment**! This code implements a **two-agent energy market simulation** where an ISO (Independent System Operator) and PCS learn to interact in an alternating training framework.

### The Two Agents

**1. ISO Agent (Grid Operator)**

- **Goal**: Set electricity prices that maximize revenue while maintaining grid stability  
- **Learns**: Optimal pricing strategies based on predicted consumption patterns  
- **Action Space**: 96 values per day (48 prices + 48 dispatch targets)  
- **Reward**: Money earned from electricity sales minus shortage penalties  

**2. PCS Agent (Household/Battery System)**

- **Goal**: Minimize electricity costs while meeting energy needs  
- **Learns**: When to use battery storage vs. grid power based on prices  
- **Action Space**: Battery charge/discharge decisions  
- **Reward**: Cost savings from strategic energy management  

### How They Interact
For each iteration:

1. **ISO Phase**: Train the grid operator for N days  
   - ISO sets prices based on current policy  
   - Observes household responses  
   - Updates pricing strategy  

2. **PCS Phase**: Train households for the next N days  
   - Households respond to ISO's prices  
   - Learn optimal battery strategies  
   - Update energy management policy  

This mimics real-world dynamics where grid operators and consumers continuously adapt to each other's behavior.

## Key Components

- **AlternatingISOEnv**: Coordinates both agents:  
  - Runs PCS environment for 48 half-hour steps (one day)  
  - Collects metrics: money earned, forecast accuracy (MAE), shortages  
  - Passes observations between the two systems  
- **PenalizedAlternatingISOEnv**: Adds financial penalties for shortages (supply < demand) to encourage conservative dispatch decisions. The effects of this strategy have yet to be studied in this project.  
- **RLPriceCurveStrategy**: Converts ISO actions into price curves that the PCS agent observes  



Let's see an example! First, let's important the relevant code.

In [ ]:
import sys
sys.path.append("/content/energy-net/energy_net/gym_envs")
from energy_net.gym_envs.alternating_env import *
from energy_net.gym_envs.evaluate_alternating import *


Next, let's run a short loop.  

All you need to do is call `run_alternating_training`. If you want to keep track of the training process for later visualization, you can assign the output to a variable (here, `history`): history = run_alternating_training(...)


### Parameters

- **cycle_days**: Number of days per training cycle. Determines how long each agent has to learn a policy before switching control to the other agent.  
  *Example:* 7 → agents switch control every week.

- **total_iterations**: Total number of training iterations.

- **pcs_algo_cls**: Algorithm class for the PCS agent. Default: `PPO`.

- **iso_algo_cls**: Algorithm class for the ISO agent. Default: `PPO`.

- **pcs_policy**: Policy type for the PCS agent.

- **iso_policy**: Policy type for the ISO agent.

- **pcs_algo_kwargs**: Additional keyword arguments for the PCS algorithm.

- **iso_algo_kwargs**: Additional keyword arguments for the ISO algorithm.

- **pcs_steps_per_day**: Number of steps per day for PCS training.

- **test_data_file**: Path to the test data CSV file.

- **predictions_file**: Path to the predictions CSV file.

- **verbose**: Verbosity level.

- **render**: Whether to render the environment.

- **render_every_n_steps**: Render frequency.


In [ ]:
warnings.simplefilter("ignore")
history = run_alternating_training(
    cycle_days=7,
    total_iterations=30,
    render=False,              # Show daily reports
    render_every_n_steps=100,    # Render every 100 timesteps
    test_data_file = 'tests/gym/data_for_tests/synthetic_household_consumption_test.csv',
    predictions_file = 'tests/gym/data_for_tests/consumption_predictions.csv',
)

Now, if we want to run an experiment and see visual results, we can use run_experiment for visualization.

In [ ]:
# Run normally
metrics, results_df = run_experiment(
    actual_csv="tests/gym/data_for_tests/synthetic_household_consumption_test.csv",
    pred_csv="tests/gym/data_for_tests/consumption_predictions.csv",
    iterations=90,
    num_days=21,
    out_dir="/content/results",
    run_id="colab_test",
)

# Then display saved files
import glob
from IPython.display import Image, display

files = sorted(glob.glob("/content/results/*.png"))
print("Found images:", files)
for f in files:
    display(Image(filename=f))


### Challenges
- Achieving convergence has proven difficult. While the PCS agent’s policy tends to converge reasonably well in isolation, the simultaneous adaptation of the ISO agent appears to disrupt this process.  
- Establishing an effective reward-sharing mechanism between the two agents. Although we experimented with an environment designed to encourage collaboration, the impact of this approach has not yet been fully evaluated.  
- The PCS agent exhibits a consistent reluctance to purchase substantial amounts of energy in these scenarios, which may prevent short-term financial losses but ultimately exacerbates its situation by generating significant shortages.
### Potential Approaches for Optimization
- Enhance the ISO agent’s ability to leverage the provided prediction data more effectively.  
- Allow additional training time for both agents to facilitate better policy development.  
- Experiment with alternative learning algorithms to improve convergence and coordination.  
- Test different cycle lengths to provide each agent with sufficient time to adapt.  
- Develop and implement a more effective shared reward structure to encourage collaboration between PCS and ISO.
